In [8]:
#Importing the necessary libraries
import pandas as pd
from sklearn import preprocessing
import numpy as np
from math import exp
from sklearn.cross_validation import KFold
from sklearn.preprocessing import PolynomialFeatures
#Importing the Iris dataset.
data_df = pd.read_csv('iris.dat',header= None)
data_df.columns = ['A', 'B', 'C', 'D', 'E']
#Creating the 2 class dataset
two_class = data_df[((data_df.E=='Iris-setosa') == True) | ((data_df.E== 'Iris-versicolor') == True)]
from math import log

In [9]:
#Applying Hot encoding.
label_encoder = preprocessing.LabelEncoder()

In [10]:
two_class.head()
two_class['E'] = label_encoder.fit_transform(two_class['E'])

/root/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [11]:
two_class.tail()

,A,B,C,D,E
95,5.7,3.0,4.2,1.2,1
96,5.7,2.9,4.2,1.3,1
97,6.2,2.9,4.3,1.3,1
98,5.1,2.5,3.0,1.1,1
99,5.7,2.8,4.1,1.3,1


In [12]:
X_2class = np.asarray(two_class.iloc[:,:two_class.shape[1]-1])
y_2class = np.asarray(two_class['E'])

In [13]:
#Function for measuring the accuracy.
def accuracy(predicted,actual):
    tot = 0
    for i in range(len(predicted)):
        if predicted[i] == actual[i]:
            tot += 1
    acc = round(1. *tot/len(predicted),5)
    #print "Accuracy: ",acc
    return acc

In [14]:
#Function for classification.
def classify(param,X):
    y_predicted = []
    theta_t =  np.transpose(param)
    for i in range(len(X)):
        x =  np.transpose(X[i]).reshape(len(X[i]),1)
        sigmoid =  1.*1/(1+exp(-(np.dot(theta_t,x))))
        if sigmoid < 0.5 :
            y_predicted.append(0)
        else:
            y_predicted.append(1)

    return y_predicted

In [15]:
def create_final(y_predicted,y_actual):
    final_predicted = []
    final_actual = []
    for i in range(len(y_predicted)):
        for j in range(len(y_predicted[i])):
            final_predicted.append(y_predicted[i][j])
            final_actual.append(y_actual[i][j])
    return(final_predicted,final_actual)

In [16]:
#Function for calculating the error
def calculate_error(theta,X,y):
    theta_t =  np.transpose(theta)
    x =  np.transpose(X).reshape(len(X),1)
    theta_tx = np.dot(theta_t,x)
    
    sigmoid =  1.*1/(1+exp(-(theta_tx)))
    return (sigmoid-y)*theta_tx

In [17]:
#Function used as a substitute for indicator function.
def calculate_relevance(theta,X,y):
    theta_t =  np.transpose(theta)
    x =  np.transpose(X).reshape(len(X),1)
    #print np.dot(theta_t,x)
    sigmoid =  1.*1/(1+exp(-(np.dot(theta_t,x))))
    #print (sigmoid-y)*X
    return ((sigmoid-y)*X)

In [18]:
 def gradient_descent(X,y,threshold=0.0001,learn_rt=0.001): 
        #Setting the initial convergence condition as False
        converge = False
        theta = np.zeros(shape=(X.shape[1],1))
        iterations = 0
        #print theta
        while converge ==False:
            #print "s"
            theta0 = theta.copy()
            #Calculating the Gradient
            grad = []
            for i in range(len(X)):
                grad.append(calculate_relevance(theta0,X[i],y[i]))
            gradient = [list(i) for i in zip(*grad)]
            summ = [sum(i) for i in gradient]
            theta =  np.transpose((np.transpose(theta0)[0]) - np.dot(learn_rt,summ))
            
            err_theta0 = []
            err_theta = []
            for i in range(len(X)):
                err_theta0.append(calculate_error(theta0,X[i],y[i]))
            for i in range(len(X)):
                err_theta.append(calculate_error(theta,X[i],y[i]))
            iterations += 1
            if (iterations == 200 or abs(sum(err_theta)-sum(err_theta0)) < threshold):
                converge = True
        return theta

In [23]:
from sklearn.preprocessing import PolynomialFeatures
#Doing cross validation for 2 class logistic regression
def cross_validation(X, y, n_folds=10,poly = 1):
        poly_degree = PolynomialFeatures(poly)
        cv = KFold(len(y), n_folds,shuffle = True)
        fold_no = 0
        kfold_accuracy = []
        train_error = []
        test_error = []
        results = []
        actual = []
        for train_ind, test_ind in cv:
            #Calling module for parameter calculation(X_train,y_train)
            #Calling module for classification(X_test,y_test)
            Z_train = poly_degree.fit_transform(X[train_ind])
            Z_test = poly_degree.fit_transform(X[test_ind])
            param = gradient_descent(Z_train,y[train_ind])
            
            results.append(classify(param,Z_test))
            actual.append(y[test_ind])
        final = create_final(results,actual)
        print "Accuracy of the Logistic Regression 2-class classification is %.2f percent." %(accuracy(final[0],final[1])*100)
        return (final)
Cross_val = cross_validation(X_2class,y_2class)     

Accuracy of the Logistic Regression 2-class classification is 97.00 percent.


In [24]:
#Logistic Regression for 3-class classification.
data_df['E'] = label_encoder.fit_transform(data_df['E'])
X_3class = np.asarray(data_df.iloc[:,:data_df.shape[1]-1])
y_3class = np.asarray(data_df['E'])

In [25]:
#Indicator function fro 3 class Logistic regression.
def calculate_relevance_3class(theta_vec,theta,X,y,cls,classes,typ = 'likelihood'):
    
    x =  np.transpose(X).reshape(len(X),1)
    num = exp(np.dot(theta,x))
    tot = 0
    for i in range(len(classes)):
        tot += exp(np.dot(theta_vec[i],x))
    if num == 0:
        h_theta = 0
        
    else:
        h_theta = 1.*num/tot
    grad = (h_theta - 1) * X
    if typ!= 'likelihood':
        return grad
    else:
        return h_theta

In [26]:
#Doing classification for 3 class logistic regression.
def do_classification(X,theta,classes):
    y_predicted = []
    for i in range(len(X)):
        y_hat = []
        for j in range(len(classes)):
            t= theta[j].reshape(1,len(theta[j]))
            x= X[i].reshape(len(X[j]),1)
            y_hat.append(np.dot(t,x))
        index = y_hat.index(max(y_hat))
        y_predicted.append(index)
    return y_predicted

In [27]:
#Function for calculating Log Likelihood.
def calc_log_likelihood(T,X,y,classes):
    #Calculating Log Likelihood.  
    likelihood = 0
    for cls in range(len(classes)):
        for j in range(len(X)):
            if y[j] == cls:
                likelihood += log(calculate_relevance_3class(T,T[cls],X[j],y[j],cls,classes,typ = 'likelihood'))
    return likelihood

In [28]:
#Gradient descent for 3 class logistic regression
def gradient_descent_3class(classes,X,y,threshold=0.01,learn_rt=0.00001): 
        #Setting the initial convergence condition as False
        converge = False
        iterations = 0
        theta_vec = np.zeros(shape=(len(classes),X.shape[1]))
        while converge == False:
            t0 = theta_vec.copy()
        #for cls in range(len(classes)):
            #while converge == False:
            for cls in range(len(classes)):
                theta0 = theta_vec[cls]
                #Calculating the Gradient
                grad = []
                for i in range(len(X)):
                    if y[i] == cls:
                        grad.append(calculate_relevance_3class(theta_vec,theta_vec[cls],X[i],y[i],cls,classes,typ='gradient'))
                    else:
                        pass
                gradient = [list(i) for i in zip(*grad)]
                
                summ = [sum(i) for i in gradient]
                
                #theta_vec[cls] =  theta0 - np.dot(learn_rt,summ)
                subtr = np.dot(learn_rt,summ)
                theta_vec[cls] = [theta0[i] - subtr[i] for i in range(len(subtr))]
            t1 = theta_vec
            #Calculating Difference of the Previous and the Current Log Likelihood.
            delta = calc_log_likelihood(t1,X,y,classes) - calc_log_likelihood(t0,X,y,classes)
            iterations += 1
            if (iterations == 185 ):
                converge = True
       
        return theta_vec

In [29]:
from sklearn.preprocessing import PolynomialFeatures
def cross_validation_3class(X, y, n_folds=10,poly = 2):
        poly_degree = PolynomialFeatures(poly)
        cv = KFold(len(y), n_folds,shuffle = True)
        fold_no = 0
        data_df.drop_duplicates(cols = 'E', inplace = True)
        classes = data_df['E'].tolist()
        kfold_accuracy = []
        train_error = []
        test_error = []
        results = []
        actual = []
        for train_ind, test_ind in cv:
            #Calling module for parameter calculation(X_train,y_train)
            #Calling module for classification(X_test,y_test)
            Z_train = poly_degree.fit_transform(X[train_ind])
            Z_test = poly_degree.fit_transform(X[test_ind])
            param = gradient_descent_3class(classes,Z_train,y[train_ind])
            results.append(do_classification(Z_test,param,classes))
            actual.append(y[test_ind])
        final = create_final(results,actual)
        print "Accuracy of the Logistic Regression 3-class classification is %.2f percent." %(accuracy(final[0],final[1])*100)
Call_cross_validation = cross_validation_3class(X_3class,y_3class)

Accuracy of the Logistic Regression 3-class classification is 77.33 percent.


/root/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: FutureWarning: the 'cols' keyword is deprecated, use 'subset' instead


In [30]:
  #========Neural Networks==============

In [31]:
#For 3 class Neural Networks for 2 layer feedforward Multi Layer Perceptron.
import numpy as np
from sklearn.datasets import fetch_mldata
import numpy as np
from sklearn.cross_validation import KFold
import os
from sklearn.preprocessing import PolynomialFeatures
from pylab import *
%matplotlib inline
mnist = fetch_mldata('MNIST original')
#print mnist.data.shape
#print mnist.target.shape
np.unique(mnist.target)

X, y = mnist.data / 255., mnist.target
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]
X_train.shape
y_train.shape
size=len(y_train)

## extract digits '0' and '1' for 2 class Logistic regression."
ind_2class = [ k for k in range(size) if y_train[k]==0 or y_train[k] == 1 or y_train[k] == 2]
extracted_images_2class=X_train[ind_2class,:]
y_actual_3class = y_train[ind_2class]
#print extracted_images_2class.shape

X_3class = np.asarray(extracted_images_2class)
print (X_3class).shape,len(y_actual_3class)
#print X_2class
#mean_image=extracted_images.mean(axis=0)
#imshow(mean_image.reshape(28,28), cmap=cm.gray)
#show()

(18623, 784) 18623


In [32]:
#Selecting random records for further analysis
idx = np.random.choice(np.arange(len(X_3class)),500, replace=False)
X_3class = X_3class[idx]
y_actual_3class = y_actual_3class[idx]

In [33]:
#Calculating Z
def calculate_zi(W,X):
    Z_i = []
    
    x = X.reshape(len(X),1)
   
    for i in range(len(W)):
        w = W[i].reshape(1,len(W[i]))
        Z_i.append((1.*1/(1+exp(-(np.dot(w[0],x))))).tolist()[0])
        
    return Z_i

In [34]:
#Calculating the predicted value using softmax function
def calculate_y_ji_hat(V,Vj,Zi):
    y_hat = []
    den = 0
    Vj = Vj.reshape(1,len(Vj))
    #Calculating denominator of softmax function.
    for i in range(len(V)):
        vc = V[i].reshape(1,len(V[i]))
        den += exp(np.dot(vc,Zi))
    return 1.*exp(np.dot(Vj,Zi))/den #Returning the softmax function value. 

In [35]:
def calc_params_nn(X,y,classes,poly_degree,h=200,lr = 0.00001,threshold = 0):
    #Initial guess of Vj and Wj = 0.
    #V Dimensions --> Num of classes * number of hidden units.
    V_vec = np.zeros(shape=(len(classes),h+1))
    #W Dimensions --> Num of Hidde Units * Number of features.
    W_vec = np.zeros(shape = (h,len(X[0])))   
    iterations = 0
    converge = False
    while converge == False:
        V_vec_prev = V_vec.copy()
        W_vec_prev = W_vec.copy()
        #Calclulation of Zi and yi hat.
        #z_i = []
        #y_ihat = []
        #Calculating Zi and y_ji_hat.
        #print poly_degree
        for cls in range(len(classes)):
            grad = 0
            for i in range(len(X)):
                if y[i] == cls:
                    z_arr = calculate_zi(W_vec_prev,X[i])
                    z_i = poly_degree.fit_transform(z_arr)[0]
                    y_ihat = (calculate_y_ji_hat(V_vec_prev,V_vec_prev[cls],z_i)) 
                    grad += (y_ihat - 1)*z_i
            #Update equation for V.
            V_vec[cls] =  V_vec_prev[cls] - lr*grad
            
        #Updating W.
        for i in range(len(W_vec_prev)):
            final_grad = 0
            for j in range(len(X)):
                grad = 0
                z_arr = calculate_zi(W_vec_prev,X[j])
                z_i = poly_degree.fit_transform(z_arr)[0]
                for cls in range(len(classes)):
                    y_ihat = (calculate_y_ji_hat(V_vec,V_vec[cls],z_i))
                    grad += (y_ihat - y[j])*V_vec[cls][i+1]
                final_grad += grad*z_arr[i]*(1-z_arr[i])*X[j]
            W_vec[i] = W_vec_prev[i] - lr*final_grad
        iterations += 1
        print iterations
        if (iterations == 50):
            converge = True
            return V_vec,W_vec

In [36]:
def do_classification(V,W,X,classes,poly_degree,h=10):
    results = []
    for i in range(len(X)):
        y_ihat = []
        for j in range(len(classes)):
                    z_arr = calculate_zi(W,X[i])
                    z_i = poly_degree.fit_transform(z_arr)[0]
                    y_ihat.append(calculate_y_ji_hat(V,V[j],z_i)) 
        results.append(y_ihat.index(max(y_ihat)))
    return results
                   

In [37]:
def cross_validation_nn(X, y,n_folds=10,poly = 1):
        poly_degree = PolynomialFeatures(poly)
        cv = KFold(len(y), n_folds,shuffle = True)
        fold_no = 0
        kfold_accuracy = []
        train_error = []
        test_error = []
        classes = list(set(y))
        for train_ind, test_ind in cv:
            #Calling module for parameter calculation(X_train,y_train)
            #Calling module for classification(X_test,y_test)
            Z_train = poly_degree.fit_transform(X[train_ind])
            Z_test = poly_degree.fit_transform(X[test_ind])
            params = calc_params_nn(Z_train,y[train_ind],classes,poly_degree,h=10,threshold = 0)
            #print y[test_ind]
            results = do_classification(params[0],params[1],Z_test,classes,poly_degree)
            #print results
            actual = y[test_ind].tolist()
            print "Accuracy of Neural-Networks 3-class classification is %.2f." %accuracy(Cross_val[0],Cross_val[1])   
            break
cross_validation_nn(X_3class,y_actual_3class)

Accuracy of Neural-Networks 3-class classification is 0.97.
